In [1]:
## import all necessary libraries ##

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import copy

#########################

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.log import case_statistics
from pm4py.objects.log.util import insert_classifier
from pm4py.util import constants

#########################

import distance

from similarity.levenshtein import Levenshtein
levenshtein = Levenshtein()

from similarity.damerau import Damerau
damerau = Damerau()

from pyjarowinkler import distance as jwdistance
from similarity.jarowinkler import JaroWinkler
jarowinkler = JaroWinkler()

from similarity.weighted_levenshtein import WeightedLevenshtein
from similarity.weighted_levenshtein import CharacterSubstitutionInterface
import math
from random import sample
from random import seed
class CharacterSubstitution(CharacterSubstitutionInterface):
    def cost(self, c0, c1):
        return math.inf # assign inifte weight to all substitutions
levenshtein2 = WeightedLevenshtein(CharacterSubstitution())

#########################

import pomegranate as pom
from sklearn import model_selection as ms

# LOF

## Distance class

In [2]:
class Distance:
    
    ### SETUP ###
    
    ## load dataset, generate mapping and generate strings
    def __init__(self, log):
        self.log = log
        
        self.strings = []
        self.matrix = []
        self.transl = {}
        self.variant_to_Vindex = {}
        self.index_to_variant = [] 
        #seed(1633048)
        #self.log = sample(self.log, int(len(self.log)/10))
        self.clear_caches()
        
        self.gen_trace_to_Tindex()
        
        self.gen_mapping()
        self.gen_variant_strings()        
    
    def clear_caches(self):
        self.Nk_res_dict = {} # N_k result cache
    
    ## generate mapping from activity to char
    def gen_mapping(self):
        ## generate mapping from activities to chars ##
        # TODO read Activity Classifier for correct naming of activities
        #activities = list(attributes_filter.get_attribute_values(self.log, "concept:name").keys())
        activities = list(attributes_filter.get_attribute_values(self.log, "customClassifier").keys())
        #activities2 = list(attributes_filter.get_attribute_values(self.log, "lifecycle:transition").keys())
        #activities = [i + "-" + j for i, j in zip(activities2, activities2)]
        for i, a in enumerate(activities):
            self.transl[a] = chr(i+1)

    def gen_trace_to_Tindex(self):
        self.trace_to_Tindex = {}
        for i, t in enumerate(self.log):
            self.trace_to_Tindex[t] = i
            
            
    ## generate strings for all variants
    def gen_variant_strings(self):
        self.variants = variants_filter.get_variants(self.log, parameters={variants_filter.Parameters.ACTIVITY_KEY: "customClassifier"}) # all variants as dictionary
        variant_strings = list(self.variants.keys()) # variants as strings
        self.variant_to_index = {} # dictionary to translate variant to index in list for later lookup of traces
        
        for i, v in enumerate(variant_strings):
            string = ""
            for e in v.split(","):
                string = string + self.transl[e] 
            
            #self.strings.append(list_to_string(v.split(",")))
            self.strings.append(string)
            
            self.variant_to_index[v] = i
            self.index_to_variant.append(v)
            
        print("Number of variants: " + str(len(self.strings)))
    
    ### CALCULATION ###
    
    ## calculate distance matrix
    def calculate(self):
        n = len(self.strings)
        self.matrix = np.full((n, n), 0, dtype = np.uint8)

        for i, x in enumerate(self.strings):
            for j, y in enumerate(self.strings):
                if j >= i: # only calculate upper right triangle of matrix
                    #dist = distance.hamming(x, y)
                    dist = levenshtein.distance(x, y)
                    #dist = levenshtein2.distance(x, y)
                    #dist = damerau.distance(x, y)
                    #dist = (1- jarowinkler.similarity(x, y))*255
                    #print(dist)
                    self.matrix[i][j] = dist

        # mirror upper right triangle of matrix by adding the transposition
        self.matrix = self.matrix + self.matrix.T

        return self.matrix
    
    ### RETRIEVAL ###
    
    ## translate trace to its corresponding matrix index
    def trace_to_index(self, trace):
        # convert trace to string tion of variant (concept:name separated by commas)
        trace_string = ""
        for e in trace:
            #trace_string = trace_string + e["concept:name"] + ","
            trace_string = trace_string + e["customClassifier"] + ","
        trace_string = trace_string[:-1] # remove last comma

        return self.variant_to_index[trace_string]
    
    ## translate matrix (variant) index to trace indices
    def index_to_traces(self, i):
        variant_string = self.index_to_variant[i]
        traces = self.variants[variant_string] # retrieve traces from variant dictionary
        filtered_variants = {variant_string: traces} # generate new variants dictionary with only one variant
        
        filtered_log = variants_filter.apply(self.log, filtered_variants, parameters={variants_filter.Parameters.ACTIVITY_KEY: "customClassifier"})
        traces = []
        for t in filtered_log:
            traces.append(t)
        
        return traces
        
    
    ## retrieve distance of two traces from matrix
    def dist(self, t1, t2):
        i1 = self.trace_to_index(t1)
        i2 = self.trace_to_index(t2)
        return self.matrix[i1][i2]
    
    # return traces of k nearest neighbors of A
    def N_k(self, k, A):
        A_variant_index = self.trace_to_index(A)
        if A_variant_index in self.Nk_res_dict.keys(): # check result cache
            #print("N_k cache hit")
            return self.Nk_res_dict[A_variant_index]
        else:
            idx_sort = np.argsort(self.matrix[A_variant_index]) # indices of neighbors in ascending distance

            i = 0
            N_k_traces = []
            while len(N_k_traces) < k:
                N_k_traces.extend(self.index_to_traces(idx_sort[i]))
                if A in N_k_traces:
                    N_k_traces.remove(A)
                i = i + 1
                
            
            self.Nk_res_dict[A_variant_index] = N_k_traces
            return N_k_traces

## LOF class

In [3]:
class LOF:
    
    def __init__(self, log):
        self.dist = Distance(log)
        self.dist.calculate()
        self.clear_caches()
    
    def clear_caches(self):
        self.dist.clear_caches()
        
        # result caches
        self.kd_res_dict = {}
        self.rd_res_dict = {}
        self.lof_res_dict = {}
        self.lrd_res_dict = {}
    
    
    ## k-distance
    def k_distance(self, k, A):
        A_variant_index = self.dist.trace_to_index(A)
        if A_variant_index in self.kd_res_dict.keys(): # check result cache
            #print("k_distance cache hit")
            return self.kd_res_dict[A_variant_index]
        else:
            N_k = self.dist.N_k(k, A)
            k_variant_index = self.dist.trace_to_index(N_k[-1])
            A_variant_index = self.dist.trace_to_index(A)

            res = self.dist.matrix[A_variant_index][k_variant_index] # retrieve distance from k-th nearest neighbor (-1 to offset arraystart, +1 to not include trace itself)
            self.kd_res_dict[A_variant_index] = res
            return res
        
    ## reachability distance
    def reachability_dist(self, k, A, B):
        A_variant_index = self.dist.trace_to_index(A)
        B_variant_index = self.dist.trace_to_index(B)
        if (A_variant_index, B_variant_index) in self.rd_res_dict.keys(): # check result cache
            #print("rd cache hit")
            return self.rd_res_dict[(A_variant_index, B_variant_index)]
        else:
            res = max(self.k_distance(k, B), self.dist.matrix[A_variant_index][B_variant_index])
            self.rd_res_dict[(A_variant_index, B_variant_index)] = res
            return res
    
    ## local reachability density
    def lrd(self, k, A):
        A_variant_index = self.dist.trace_to_index(A)
        if A_variant_index in self.lrd_res_dict.keys(): # check result cache
            #print("lrd cache hit")
            return self.lrd_res_dict[A_variant_index]
        else:
            
            N_k = self.dist.N_k(k, A)            
            sum = 0
            for b in N_k:
                   sum = sum + self.reachability_dist(k, A, b) # sum of rachability distances in k-Neighborhood
            
            res = 1 / (sum / len(N_k))
            self.lrd_res_dict[A_variant_index] = res
            return res
        
    def lof(self, k, A):
        A_variant_index = self.dist.trace_to_index(A)
        if A_variant_index in self.lof_res_dict.keys(): # check result cache
            #print("lof cache hit")
            return self.lof_res_dict[A_variant_index]
        else:
            
            N_k = self.dist.N_k(k, A)
            sum = 0
            for b in N_k:
                sum = sum + self.lrd(k, b)

            res = sum / (len(N_k) * self.lrd(k, A))
            self.lof_res_dict[A_variant_index] = res
            return res
    
    def calculate(self, k):
        res = np.array([])
        
        for a in self.dist.log:
            res = np.append(res, self.lof(k, a))
            
        return res
    

# Run

## Import log

In [49]:
path = "Datasets/BPIC20_sample.xes"
log = xes_importer.apply(path)

# generate custom activity classifier
try:
    
    #log, activity_key = insert_classifier.insert_activity_classifier_attribute(self.log, "Activity classifier")
    for trace in log:
        for event in trace:
            custom_classifier = ""
            for activity_classifier in log.classifiers["Activity classifier"]:
                custom_classifier = custom_classifier + event[activity_classifier] + "+"
            custom_classifier = custom_classifier[:-1]
            event["customClassifier"] = custom_classifier
except:
    print("foo")
    for trace in log:
        for event in trace:
            event["customClassifier"] = event["concept:name"]

# generate index attribute for each event (later used to fiter)
for trace in log:
    for i, event in enumerate(trace):
        event["index"] = i


parsing log, completed traces ::   0%|          | 0/353 [00:00<?, ?it/s]

foo


## Generate LOFs

In [6]:
start = datetime.now()
print(start)

# find length of longest trace
trace_len = [len(i) for i in log]
max_trace_len = max(trace_len)
print("max trace length: " + str(max_trace_len))

res = []

for l in range(1, max_trace_len+1): # iterate from 1 to length of longest trace
    print("l: " + str(l))
    # filter events by attribute "index". only events with index between 0 and l are kept
    log_tmp = attributes_filter.apply_numeric_events(log, 0, l, 
                                                     parameters={constants.PARAMETER_CONSTANT_ATTRIBUTE_KEY: "index"})
    
    # run LOF calculation on filtered log
    lof = LOF(log_tmp)

    k = len(max(lof.dist.variants.values(), key=len)) + 1 # no. traces in largest variant + 1
    res_tmp = lof.calculate(k)
    res.append(res_tmp)
    
end = datetime.now()
print(end-start)

2021-06-29 09:29:46.992670
max trace length: 35
l: 1
Number of variants: 21
l: 2
Number of variants: 49
l: 3
Number of variants: 107
l: 4
Number of variants: 184
l: 5
Number of variants: 248
l: 6
Number of variants: 290
l: 7
Number of variants: 314
l: 8
Number of variants: 322
l: 9
Number of variants: 325
l: 10
Number of variants: 326
l: 11
Number of variants: 327
l: 12
Number of variants: 327
l: 13
Number of variants: 327
l: 14
Number of variants: 327
l: 15
Number of variants: 327
l: 16
Number of variants: 327
l: 17
Number of variants: 327
l: 18
Number of variants: 327
l: 19
Number of variants: 327
l: 20
Number of variants: 327
l: 21
Number of variants: 327
l: 22
Number of variants: 327
l: 23
Number of variants: 327
l: 24
Number of variants: 327
l: 25
Number of variants: 327
l: 26
Number of variants: 327
l: 27
Number of variants: 327
l: 28
Number of variants: 327
l: 29
Number of variants: 327
l: 30
Number of variants: 327
l: 31
Number of variants: 327
l: 32
Number of variants: 327
l: 

In [7]:
# mirror results and rotate by 270 degrees
res_rot = np.rot90(res[::-1], 3) 

In [50]:
res_rot = np.genfromtxt("res_BPIC20_sample.csv", delimiter=",")
res = np.rot90(res_rot)
res = res[::-1]

In [51]:
res.shape

(41, 353)

In [52]:
# define cutoffs for anomaly at each length
cutoffs95 = []
cutoffs85 = []
cutoffs50 = []
for l in range(len(res)):
    r = []
    for t in range(len(res_rot)):
        if len(log[t]) >= l + 1:
            r.append(res[l][t])
    cutoffs50.append(np.percentile(r, 50))

In [53]:
# classify traces at each point
classification = [None] * len(res_rot)

for j, trace_res in enumerate(res_rot):
    classification_trace = [None] * len(log[j])
    for i, r in enumerate(trace_res):
        if i < len(log[j]):
            if r >= cutoffs50[i]:
                classification_trace[i] = "anomaly"
            else:
                classification_trace[i] = "no anomaly"
    classification[j] = classification_trace

In [54]:
cutoffs50

[1.0,
 1.0,
 1.23347703,
 1.64047624,
 1.44341236,
 1.36110976,
 1.58584883,
 1.24288031,
 1.16178767,
 1.18127806,
 1.7242029,
 2.29242201,
 2.2737838,
 2.49398863,
 2.37406605,
 2.35960501,
 2.35566534,
 2.59372323,
 2.695996015,
 2.66267306,
 2.79092327,
 3.419704205,
 3.22642968,
 3.3500217,
 3.4713991049999997,
 3.55468006,
 3.71630166,
 3.617872785,
 3.31841485,
 3.6782361,
 4.40117114,
 4.61862394,
 4.79048487,
 4.297790035,
 4.399185105,
 4.612745545,
 4.89125937,
 4.950587390000001,
 5.0318671550000005,
 4.24370181,
 4.24370181]

In [292]:
np.savetxt("res_BPIC20_sample.csv", res_rot, delimiter=",", fmt='%1.8f')

In [55]:
na = 0
a = 0
for t in classification:
    for c in t:
        if c == "no anomaly":
            na = na+1
        elif c == "anomaly":
            a = a+1
            
t = na + a
print(na/t)
print(a/t)

0.421878606046619
0.578121393953381


## Setup data

In [61]:
lists = []
for trace in log:
    tlist = []
    for i, event in enumerate(trace):
        tlist.append(event["customClassifier"])
    lists.append(tlist)


# split data into training and test
lists_train, lists_test, is_anomaly_train, is_anomaly_test = ms.train_test_split(lists, classification, test_size = 0.2, random_state = 3)

## Setup model
- Discrete observations
- Two states

In [62]:
model = pom.HiddenMarkovModel.from_samples(
    pom.DiscreteDistribution, 
    n_components=2,
    X=lists_train, 
    labels=is_anomaly_train,
    #state_names=["anomaly", "no anomaly", "unknown"],
    state_names=["no anomaly", "anomaly"],
    algorithm="labeled")
model.bake()

In [58]:
model.states

[{
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "numpy.str_",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "Declaration APPROVED by ADMINISTRATION" : 0.05960648148148148,
                 "Declaration APPROVED by BUDGET OWNER" : 0.02546296296296296,
                 "Declaration APPROVED by PRE_APPROVER" : 0.007523148148148148,
                 "Declaration APPROVED by SUPERVISOR" : 0.003472222222222222,
                 "Declaration FINAL_APPROVED by DIRECTOR" : 0.0028935185185185184,
                 "Declaration FINAL_APPROVED by SUPERVISOR" : 0.08043981481481481,
                 "Declaration REJECTED by ADMINISTRATION" : 0.01678240740740741,
                 "Declaration REJECTED by BUDGET OWNER" : 0.0005787037037037037,
                 "Declaration REJECTED by DIRECTOR" : 0.0005787037037037037,
                 "Declaration REJECTED by EMPLOYEE" : 0.010416666666666666,

In [59]:
model.dense_transition_matrix()

array([[0.73033708, 0.26966292, 0.        , 0.        ],
       [0.27944573, 0.72055427, 0.        , 0.        ],
       [0.5       , 0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

## Evaluate

In [63]:
false_positive = 0 # anomaly incorrectly detected
false_negative = 0 # anomaly incrrectly not detected
true_positive = 0 # anomaly correctly detectd
true_negative = 0 # anomaly correclty not detected

for i, t in enumerate(lists_test):
    prediction = model.predict(t)
    if prediction[-1] == 0 and is_anomaly_test[i][-1] == "no anomaly":
        true_negative = true_negative + 1
    if prediction[-1] == 0 and is_anomaly_test[i][-1] == "anomaly":
        false_negative = false_negative + 1
    if prediction[-1] == 1 and is_anomaly_test[i][-1] == "no anomaly":
        false_positive = false_positive + 1
    if prediction[-1] == 1 and is_anomaly_test[i][-1] == "anomaly":
        true_positive = true_positive + 1

TPR = true_positive / (true_positive + false_negative)
TNR = true_negative / (true_negative + false_positive)
        
FPR = false_positive / (false_positive + true_negative)
FNR = false_negative / (false_negative + true_positive)

In [64]:
model.predict(lists_test[1])
is_anomaly_test[1]

['anomaly', 'no anomaly', 'anomaly', 'anomaly', 'anomaly', 'no anomaly']

In [65]:
print("FPR\t\t\t" + str(round(FPR * 100, 1)) + "%")
print("FNR\t\t\t" + str(round(FNR * 100, 1)) + "%")
print("\n")
print("TNR (sensitivity)\t" + str(round(TNR * 100, 1)) + "%")
print("TPR (specificity)\t" + str(round(TPR * 100, 1)) + "%")

FPR			28.6%
FNR			88.9%


TNR (sensitivity)	71.4%
TPR (specificity)	11.1%


In [66]:
print("neg: " + str(false_positive + true_negative))
print("pos: " + str(false_negative + true_positive))


neg: 35
pos: 36
